# Building a Neural Network from Scratch Using Object-Oriented Programming (OOP)

In this notebook I develop an object-oriented sequential neural network from scratch as a personal project. The main motivator for this work is a more rigorous application of concepts learned in Andrew Ng's machine learning courses:

- [Machine Learning Specialisation](https://www.coursera.org/specializations/machine-learning-introduction)
- [Deep Learning Specialisation](https://www.coursera.org/specializations/deep-learning)


The full implementation will have two main components:


1.   A neural network layer constructor that allows the user to define layer properties similarly to the Keras API for TensorFlow.
2.   A model compiler that takes a series of layers and forms the neural network.



In [7]:
# Import packages
import numpy as np

# Neural Network Layer Object

The neural network layer object is created below. While the goal is mimic Keras' ease of use through OOP, the features included in this build are not an exhaustive reproduction of Keras' feature list, since that is well beyond the scope of this work. The following functionality  is included in the current version:
- Generalised object architechture allows the user to initialise a layer with
  - a number of units/neurons
  - an activation function
  - weight and bias parameters
  - a layer id/name
- A <u>print()</u> call produces a brief summary of the layer properties.
- The <u>initialiseWB()</u> method initialises the layer weights and biases based on an expected input.
- Once the model has been initialised for an input, the layer object validates any future inputs using the <u>is_consistent()</u> method.
- The <u>get_parameters()</u> and <u>set_parameters()</u> methods allow for the inspection and  update of the layers weights and biases to facilitate the learning mechanisms.
- The <u>activate()</u> method computes the local forward propagation step (vectorised) using one of the following activation functions:
  - Linear (i.e, no activation function)
  - Sigmoid
  - Hyperbolic Tangent Function (tanh)
  - Softmax
  - Rectified Linear Unit (ReLU)


In [110]:
class NN_DenseLayer():
  # class variables for scaling initial weights/biases
  WB_SCALE = 0.01

  def __init__(self,units,activation='sigmoid',name=None,W_init=None,B_init=None):
    # initialise layer data
    self.neurons = units          # number of neurons in the layer
    self.activation = activation  # activation function applied at the layer
    self.name = name              # layer name (if desired)

    # features will be updated when an input is passed to the layer
    self.features = None

    # initialise layer weights and biases (inactive by default)
    self.wb_initialised = False
    self.W = W_init
    self.B = B_init

    # input sanitation parameter
    self.expectedInputShape = None


  def __str__(self):
    '''
      Prints a brief summary of the layer properties when print() is called.
    '''
    # print dialogue
    return f"\nNeural-network layer (dense) with {self.neurons} neurons. {self.activation.title()} activation. \n"


  def initialiseWB(self,X_in,featureCount,neuronCount):
    '''
      Produces initial values for weight and bias arrays:

      PARAMETER INITIALISATION PROTOCOL:
      --------------------------------------------------------------------------
        - initialise the weights matrix to the desired (n,j), populating with a random sample from a normal distributiion
        - initialise the bias matrix to the desired (1,j), populating with zeros
        - scale both values using WB_SCALE
    '''
    # initialise weight array, and scale
    weights_matrix = np.random.randn(featureCount,neuronCount) * self.WB_SCALE

    # initialise bias array, and scale
    bias_matrix = np.zeros((1,neuronCount)) * self.WB_SCALE

    # indicate that weights/biases have been initialised
    self.wb_initialised = True

    return weights_matrix, bias_matrix


  def is_consistent(self,X_in):
    '''
    Checks if the input X_in is consistent with previous inputs to the layer.
    '''
    # returns true if the input shape is consistent with the previous iterations
    if X_in.shape == self.expectedInputShape:
      return True
    return False


  def update_input(self, X_in):
    '''
      Updates the expected input shape for the layer based on the structure of a new input X_in
    '''
    # update the expected input shape to match X_in
    self.expectedInputShape = X_in.shape



  def count_features(self,X_in):
    '''
    Counts the number of features, n, present in the input X_in
    '''
    try:
      # if X_in is a matrix, n is the number of columns, index 1
      numFeatures = X_in.shape[1]
    except IndexError:
      # an IndexError above would suggest that X_in is a 1D array. Therefore n is index 0
      numFeatures = X_in.shape[0]

    return numFeatures


  def activate(self, X_in):
    '''
      Evaluates the layer output(s) for an input X_in:

      X_in -  input data    (m,n)  |  m examples with n features each
      W    -  layer weights (n,j)  |  n features per neuron for j neurons/units
      B    -  bias vector   (1,j)  |  j neurons/units
    '''
    self.current_input = X_in
    # check if the layer was expecting an input of this format
    if self.is_consistent(self.current_input):
      pass # do nothing if input was expected in this format
    else:
      # we're dealing with a new input. so update the expected input shape
      self.update_input(self.current_input)
      # count the number of features in the new input format
      self.features = self.count_features(self.current_input)
      # initialise weight/bias arrays for the new input
      self.W, self.B = self.initialiseWB(self.current_input,self.features,self.neurons)

    # if weights/biases have not yet been initialised, then do so.
    if not self.wb_initialised:
      # estimate inital values for W and B
      self.W, self.B = self.initialiseWB(self.current_input,self.features,self.neurons)

    # apply weights an biases to inputs; (z = X_in*W + B)
    z = np.matmul(self.current_input, self.W) + self.B

    # library of built-in activation functions
    activation_library = {
        'linear':self.linear(z),
        'sigmoid':self.sigmoid(z),
        'tanh':self.tanh(z),
        'softmax':self.softmax(z),
        'relu':self.relu(z)
    }

    # attempt to apply the chosen activation function
    try:
      # pull the relevant activation output
      self.out = activation_library[self.activation.lower()]
    except KeyError:
      # activation call failed
      self.out = None
      # find valid activation functions
      valid_activation_keys = [key for (key, values) in activation_library.items()]
      # throw exception and recommend valid activation functions
      raise Exception(f"Invalid activation function. Pick one of the following: {valid_activation_keys}")

    return self.out


  def get_parameters(self):
    '''
      Returns the current weights and biases for the layer
    '''
    # package parameters into a dictionary and return
    self.parameters = {'W':self.W, 'B':self.B}
    return self.parameters


  def set_parameters(self,W_set,B_set):
    '''
      Overwrites the weights and biases of the current layer
    '''
    try:
      # if setpoint values are null, then W/B are set to None
      if (W_set == None) and (B_set == None):
        self.W = W_set
        self.B = B_set
        # indicate that the weights/biases have been nullified
        self.wb_initialised = False

    except ValueError:
      if (self.W == None) or (self.B == None):
        # create proxy weights/biases using the current input structure
        W_proxy, B_proxy = self.initialiseWB(self.current_input,self.features,self.neurons)
      else:
        W_proxy = self.W
        B_proxy = self.B

      # if non-null, the shape of the weight and bias structures should be consistent
      if (W_set.shape == W_proxy.shape) and (B_set.shape == B_proxy.shape):
        self.W = W_set
        self.B = B_set
      # if not, the W/B structures chosen are incompatible
      else:
        return print(f"\nINVALID COMMAND(!): \n\nLayer is only compatible with a {W_proxy.shape} weight matrix, and {B_proxy.shape} bias matrix")


  #----------------------------------------------------------------------------#
  # ACTIVATION FUNCTIONS
  #----------------------------------------------------------------------------#
  # Linear function (regression problems; negative or positive)
  def linear(self,z):
    '''
      Evaluates the linear function g on an input z

      Such that, g(z) = z
    '''
    return z


  # Sigmoid function (binary classification problems; 1 or 0, etc.)
  def sigmoid(self,z):
    '''
      Evaluates the sigmoid function g on an input z

      Where, g(z) =      1
                     ----------
                     1 + e^(-z)
    '''
    return 1/(1 + np.exp(-z))


  # Hyperbolic Tangent function (alternative to sigmoid, varies between -1 an 1 instead of 0 and 1)
  def tanh(self,z):
    '''
      Evaluates the Hyperbolic Tangent (tanh) function g on an input z

      Where, g(z) =    e^(z) - e^(-z)
                       --------------
                       e^(z) + e^(-z)
    '''
    return np.tanh(z)


  # ReLU function (regression problems; threshold utility; non-negative values only)
  def relu(self,z):
    '''
      Evaluates the Rectified Linear Unit (ReLU) function g on an input z

      Where, g(z) = 0, if (z < 0)
             g(z) = z, if (z > 0)
    '''
    return np.maximum(0,z)


  # Softmax activation (multi-class classification, i.e. mutiple dicrete choices, one correct choice per example)
  def softmax(self,z):
    '''
    Evaluates the softmax activation function g on an input z. Softmax converts
    z for all possible choice into a distribution of probabilities that each
    choice is correct.

    For a problem with N choices, the probability that the kth choice is
    correct is represented by:

           g(z_{i,j,k,...,N}) =         e^z_k
                                 -------------------
                                 e^z_i + ... + e^z_N
    '''
    return np.exp(z)/np.sum(np.exp(z))


# Function Tests - Layer Behaviour

Below are some tests to ensure that the neural network layer produces the expected behaviour.

In [95]:
# create a new dense layer
L1 = NN_DenseLayer(3,'Sigmoid','layer1')
# create another dense layer with a bogus activation function
L2 = NN_DenseLayer(3,'Bogus','layer2')

In [96]:
# print new layer
print(L1)


Neural-network layer (dense) with 3 neurons. Sigmoid activation. 



In [97]:
# check layer weights
L1.get_parameters()

{'W': None, 'B': None}

In [98]:
# Check the expected input shape
L1.expectedInputShape

In [99]:
# activate the layer L1 by passing an array input
test_input = np.array([[1,2,3]])
L1.activate(test_input)

array([[0.50542075, 0.50019414, 0.52050466]])

In [100]:
# summary of L2 layer
print(L2)


Neural-network layer (dense) with 3 neurons. Bogus activation. 



In [ ]:
# attempt to activate L2 using the same input. input validation should catch bogus activation
L2.activate(test_input)

In [101]:
# The expected input shape should now match the most recent input (1D array)
L1.expectedInputShape

(1, 3)

In [102]:
# check layer weights
L1.get_parameters()

{'W': array([[-0.00165777, -0.00157456,  0.00859604],
        [-0.00418253, -0.0061997 , -0.00519123],
        [ 0.01056889,  0.00491683,  0.02795036]]),
 'B': array([[0., 0., 0.]])}

In [103]:
# reset weights
W = None
B = None
L1.set_parameters(W, B)
L1.get_parameters()

{'W': None, 'B': None}

In [104]:
# set invalid weights
W = np.arange(1,3,1)
B = np.arange(1,10,2)
L1.set_parameters(W, B)


INVALID COMMAND(!): 

Layer is only compatible with a (3, 3) weight matrix, and (1, 3) bias vector


In [105]:
# check that layer weights were NOT changed
L1.get_parameters()

{'W': None, 'B': None}

In [106]:
# activate for a new input (matrix type)
new_in = np.array([[1,2,3],[4,5,6]])
L1.activate(new_in)

array([[0.49409322, 0.50071983, 0.49162361],
       [0.49121147, 0.49958555, 0.48173364]])

In [107]:
# check layer weights
L1.get_parameters()

{'W': array([[ 0.00820837, -0.00685627,  0.00587853],
        [-0.00431803,  0.00629611, -0.01783766],
        [-0.00773351, -0.00095221, -0.0012373 ]]),
 'B': array([[0., 0., 0.]])}

In [108]:
# The expected input should update to match the new input type (2D matrix)
L1.expectedInputShape

(2, 3)

# Neural Network Model Compiler

In [ ]:
class NN_Model:
  def __init__(self,layers):
    # assign layer identifiers for the NN
    self.layers = layers
    self.num_layers = len(self.layers)
    self.current_input = None


  def summarise(self):
    '''
    Summarises the features of the Neural Netwrok instance
    '''
    print(f"{len(self.layers)} Layer Neural-Network (Sequential):")
    print(f"------------------------------------\n")
    for id in range(len(self.layers)):
      print(f"Layer {id + 1}")
      print(self.layers[id])


  def compile(self):
    pass


  def fit(self):
    pass


  def forward_propagation(self):
    pass


  def backward_propagation(self):
    pass


  def gradient_descent(self):
    pass


  def update_parameters(self):
    pass


  def predict(self):
    pass



# Unit Tests - Sequential NN Behaviour

In [ ]:
# create NN layer objects
L1 = NN_DenseLayer(25,'sigmoid','layer1')
L2 = NN_DenseLayer(15,'sigmoid','layer2')
L3 = NN_DenseLayer(1,'sigmoid','layer3')

# pass layers to NN sequantial constructor model
model = NN_Model([L1,L2,L3])

In [ ]:
# model
model.summarise()

3 Layer Neural-Network (Sequential):
------------------------------------

Layer 1

Neural-network layer (dense) with 25 neurons. Sigmoid activation. 

Layer 2

Neural-network layer (dense) with 15 neurons. Sigmoid activation. 

Layer 3

Neural-network layer (dense) with 1 neurons. Sigmoid activation. 

